In [2]:
# !pip install pymongo[srv]
# !pip install sentence-transformers
# !pip install langchain_openai
# !pip install langchain-community
# !pip install langchain huggingface_hub

In [1]:
from pymongo import MongoClient
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
from langchain.vectorstores import MongoDBAtlasVectorSearch
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFaceHub
from langchain.prompts import PromptTemplate
import os

import warnings
warnings.filterwarnings("ignore")

from dotenv import load_dotenv
load_dotenv(dotenv_path="../.env")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
MONGO_DB_URI = os.getenv("MONGO_DB_URI")
HF_API_KEY = os.getenv("HF_API_KEY")

# Inisialisasi MongoDB

In [2]:
# Membangun koneksi dengan MongoDB
# URI mongo db silahkan anda ganti dengan URI dengan URI database mongodb anda yang telah terisi data vectorized shopee qna (cek file rag_vecdb.ipynb)
client = MongoClient(MONGO_DB_URI, tls=True)

In [3]:
# Collection yang berisi data vectorized shopee qna 
collection = client['shopee']['answer_vec']

# Inisialisasi Vector Search MongoDB

In [4]:
# %%capture

# Inisialisasi retriever vector search MongoDB
# embedding menggunakan model sentence-transformers/all-MiniLM-L6-v2
vectorStore = MongoDBAtlasVectorSearch(
    collection, HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"), index_name='vector_index'
)

# Prompt

In [5]:
# Prompt yang menjadi landasan model untuk menjawab pertanyaan
base_prompt = "Your name is ABANG (Asisten Bot Andalan Ngomongin e-commerce), You work as a customer service representative at Shopee, a leading e-commerce in Indonesia. Your responsibility is to give accurate answers to customer questions. All responses should be in Indonesian and based on data that was already given. Your responses should be polite, professional, and helpful. Don’t answer to any questions or inquiries that are not related to Shopee. And do not explain any application outside shopee"

prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template= base_prompt + "\n\nContext:\n{context}\n\nPertanyaan: {question}\nJawaban:"
)

# Model LLM HuggingFace Zephyr-7b-beta

In [6]:
# Silahkan gunakan KEY HuggingFace anda
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HF_API_KEY

# Definisikan penggunaan model text2text generator HuggingFaceH4/zephyr-7b-beta
llm_zephyr = HuggingFaceHub(
    repo_id='HuggingFaceH4/zephyr-7b-beta',
    model_kwargs={'temperature': 0.7, 'max_new_tokens': 256}
)

In [7]:
# membuat sistem tanya-jawab berbasis retrieval
qa_zephyr = RetrievalQA.from_chain_type(
    llm=llm_zephyr,
    chain_type='stuff',
    retriever=vectorStore.as_retriever(search_type='similarity', search_kwargs={'k': 3}),
    chain_type_kwargs={'prompt': prompt_template}
)

### Query Using Zephyr-7b-beta

In [8]:
def parsing_result (result):
  data_list = result['result'].split('\n')
  question = data_list[5].replace("Pertanyaan: ", "")
  answer = data_list[6].replace("Jawaban: ", "")
  return {
    "question": question,
    "answer": answer
  }

**Positive Case**

In [ ]:
question = 'Apa itu Lelang di Shopee Live?'
result = qa_zephyr.invoke(question)
result = parsing_result(result)
print(f'Pertanyaan: {result['question']}')
print(f'Jawaban: {result['answer']}')
print(' ')
print('Data: Aktivitas lelang di Shopee Live memungkinkan Anda untuk memenangkan penawaran produk yang ditetapkan oleh host/Penjual. Setelah Penjual memulai lelang, Kartu Lelang akan muncul di layar Anda. Kartu ini menunjukkan jenis lelang dan penghitung waktu mundur yang sudah ditentukan oleh Penjual sebagai batas akhir Anda untuk melakukan penawaran.')

Pertanyaan: Apa itu Lelang di Shopee Live?
Jawaban: Lelang di Shopee Live adalah sebuah fitur yang tersedia dalam Shopee Live, yang merupakan fitur streaming video yang memungkinkan penjual untuk menjual produk secara langsung kepada pelanggan melalui live stream. Lelang di Shopee Live menawarkan penawaran-penawaran kepada pelanggan dan mereka akan dapat produk dengan harga yang lebih murah dari harga asli. Fitur ini sangat populer karena memungkinkan penjual untuk melakukan demo produk, menghapus stok tinggi, dan mempercepat jualan produk mereka. Anda bisa mengakses Lelang di Shopee Live melalui menu "Lelang" di aplikasi Shopee.
 
Data: Aktivitas lelang di Shopee Live memungkinkan Anda untuk memenangkan penawaran produk yang ditetapkan oleh host/Penjual. Setelah Penjual memulai lelang, Kartu Lelang akan muncul di layar Anda. Kartu ini menunjukkan jenis lelang dan penghitung waktu mundur yang sudah ditentukan oleh Penjual sebagai batas akhir Anda untuk melakukan penawaran.


In [13]:
question = 'Bagaimana cara mengisi saldo ShopeePay melalui SeaBank?'
result = qa_zephyr.invoke(question)
result = parsing_result(result)
print(f'Pertanyaan: {result['question']}')
print(f'Jawaban: {result['answer']}')
print(' ')
print('Data: Buka Aplikasi SeaBank dan log in ke akun SeaBank > pilih Top Up E-Wallet > pilih ShopeePay, masukkan no. handphone/nama akun tujuan > pilih no. handphone/nama yang telah diisi > masukkan nominal transfer yang diinginkan > pilih Selanjutnya > periksa kembali informasi yang sudah dimasukkan > lalu pilih Konfirmasi dan masukkan PIN.')

Pertanyaan: Bagaimana cara mengisi saldo ShopeePay melalui SeaBank?
Jawaban: Untuk menghilangkan saldo ShopeePay Anda, Silahkan lakukan langkah berikut:
 
Data: Buka Aplikasi SeaBank dan log in ke akun SeaBank > pilih Top Up E-Wallet > pilih ShopeePay, masukkan no. handphone/nama akun tujuan > pilih no. handphone/nama yang telah diisi > masukkan nominal transfer yang diinginkan > pilih Selanjutnya > periksa kembali informasi yang sudah dimasukkan > lalu pilih Konfirmasi dan masukkan PIN.


**Negative Case**

In [15]:
question = 'Bisa ajari aku teknik bela diri?'
result = qa_zephyr.invoke(question)
result = parsing_result(result)
print(f'Pertanyaan: {result['question']}')
print(f'Jawaban: {result['answer']}')
print(' ')
print('Data: Tidak ada di QnA')

Pertanyaan: Bisa ajari aku teknik bela diri?
Jawaban: Mohon maaf, kami sebagai layanan pelanggan Shopee hanya menyediakan layanan e-commerce. Teknik bela diri adalah hal yang individu harus merasakan dan menghidupi sendiri. Kami menyarankan Anda untuk melacak berbagai sumber informasi dan belajar secara mandiri untuk meningkatkan kualitas Anda diri. Terima kasih.
 
Data: Tidak ada di QnA


Berdasarkan analisa kualitatif untuk hasil RAG Query dengan model `HuggingFaceH4/zephyr-7b-beta` didapatkan kesimpulan sebagai berikut:

1. **Apa itu Lelang di Shopee Live?**

 - Jawaban model LLM: 
    - Relevan ✅ : Menjelaskan apa itu Shopee Live, dan terdapat fitur lelangnya.
    - Kejelasan ✅ : Tutur bahasa yang digunakan nyambung dan dapat dimengerti manusia.
    - Kelengkapan ⚠️ : Kurang lengkap karena tidak menyebutkan teknisnya, seperti kartu lelang, dan penghitungan waktu.

2. **Bagaimana cara mengisi saldo ShopeePay melalui SeaBank?**

 - Jawaban model LLM: 
    - Relevan ❌ : Tidak relevan, jawaban model malah membahas mengenai menghilangkan saldo ShopeePay
    - Kejelasan ❌ : -
    - Kelengkapan ❌ : -

3. **Bisa ajari aku teknik bela diri?**

 - Jawaban model LLM: 
    - Relevan ✅ : Menolak permintaan dengan sopan dan menjelaskan bahwa ini di luar cakupan layanan
    - Kejelasan ✅ : Tutur bahasa yang digunakan nyambung dan dapat dimengerti manusia.

# Model LLM GPT-3.5

In [16]:
# mendefinisikan model gpt 3.5
# silahkan gunakan api key openAI anda
llm_gpt = ChatOpenAI(model="gpt-3.5-turbo",
                 openai_api_key=OPENAI_API_KEY,
                 temperature=0, max_tokens=512)

In [17]:
# membuat sistem tanya-jawab berbasis retrieval
qa_gpt = RetrievalQA.from_chain_type(
    llm=llm_gpt,
    chain_type="stuff",
    retriever=vectorStore.as_retriever(search_type="similarity", search_kwargs={"k": 3}),
    chain_type_kwargs={"prompt": prompt_template}
)

### Query Using GPT-3.5

**Positive Case**

In [21]:
question = 'Apa itu Lelang di Shopee Live?'
result = qa_gpt.invoke(question)
print(f'Pertanyaan: {result['query']}')
print(f'Jawaban: {result['result']}')
print(' ')
print('Data: Aktivitas lelang di Shopee Live memungkinkan Anda untuk memenangkan penawaran produk yang ditetapkan oleh host/Penjual. Setelah Penjual memulai lelang, Kartu Lelang akan muncul di layar Anda. Kartu ini menunjukkan jenis lelang dan penghitung waktu mundur yang sudah ditentukan oleh Penjual sebagai batas akhir Anda untuk melakukan penawaran.')

Pertanyaan: Apa itu Lelang di Shopee Live?
Jawaban: Lelang di Shopee Live adalah fitur yang memungkinkan pengguna untuk mengikuti lelang barang secara langsung melalui siaran langsung di aplikasi Shopee. Anda dapat menawar barang yang diinginkan dan menang lelang dengan harga terbaik. Jangan ragu untuk mencoba fitur menarik ini di Shopee Live!
 
Data: Aktivitas lelang di Shopee Live memungkinkan Anda untuk memenangkan penawaran produk yang ditetapkan oleh host/Penjual. Setelah Penjual memulai lelang, Kartu Lelang akan muncul di layar Anda. Kartu ini menunjukkan jenis lelang dan penghitung waktu mundur yang sudah ditentukan oleh Penjual sebagai batas akhir Anda untuk melakukan penawaran.


In [23]:
question = 'Bagaimana cara mengisi saldo ShopeePay melalui SeaBank?'
result = qa_gpt.invoke(question)
print(f'Pertanyaan: {result['query']}')
print(f'Jawaban: {result['result']}')
print(' ')
print('Data: Buka Aplikasi SeaBank dan log in ke akun SeaBank > pilih Top Up E-Wallet > pilih ShopeePay, masukkan no. handphone/nama akun tujuan > pilih no. handphone/nama yang telah diisi > masukkan nominal transfer yang diinginkan > pilih Selanjutnya > periksa kembali informasi yang sudah dimasukkan > lalu pilih Konfirmasi dan masukkan PIN.')

Pertanyaan: Bagaimana cara mengisi saldo ShopeePay melalui SeaBank?
Jawaban: Untuk mengisi saldo ShopeePay melalui SeaBank, Anda dapat mengikuti langkah-langkah berikut:
1. Buka aplikasi Shopee dan pilih menu ShopeePay.
2. Pilih opsi "Top Up" dan pilih metode pembayaran SeaBank.
3. Masukkan jumlah saldo yang ingin diisi dan ikuti petunjuk selanjutnya untuk menyelesaikan transaksi.
Pastikan Anda memiliki akun SeaBank yang terdaftar dan memiliki saldo yang mencukupi sebelum melakukan pengisian saldo ShopeePay. Jika ada pertanyaan lebih lanjut, jangan ragu untuk menghubungi layanan pelanggan SeaBank atau Shopee. Terima kasih.
 
Data: Buka Aplikasi SeaBank dan log in ke akun SeaBank > pilih Top Up E-Wallet > pilih ShopeePay, masukkan no. handphone/nama akun tujuan > pilih no. handphone/nama yang telah diisi > masukkan nominal transfer yang diinginkan > pilih Selanjutnya > periksa kembali informasi yang sudah dimasukkan > lalu pilih Konfirmasi dan masukkan PIN.


**Negative Case**

In [24]:
question = 'Bisa ajari aku teknik bela diri?'
result = qa_gpt.invoke(question)
print(f'Pertanyaan: {result['query']}')
print(f'Jawaban: {result['result']}')
print(' ')
print('Data: Tidak ada di QnA')

Pertanyaan: Bisa ajari aku teknik bela diri?
Jawaban: Maaf, saya hanya dapat membantu dengan pertanyaan yang berkaitan dengan layanan Shopee. Apakah ada yang bisa saya bantu terkait pembelian atau transaksi di Shopee?
 
Data: Tidak ada di QnA


In [25]:
question = 'Mengapa ayam menyebrang jalan?'
result = qa_gpt.invoke(question)
print(f'Pertanyaan: {result['query']}')
print(f'Jawaban: {result['result']}')
print(' ')
print('Data: Tidak ada di QnA')

Pertanyaan: Mengapa ayam menyebrang jalan?
Jawaban: Maaf, pertanyaan tersebut tidak terkait dengan layanan Shopee. Apakah ada pertanyaan lain yang bisa saya bantu?
 
Data: Tidak ada di QnA


Berdasarkan analisa kualitatif untuk hasil RAG Query dengan model `gpt-3.5-turbo` didapatkan kesimpulan sebagai berikut:

1. **Apa itu Lelang di Shopee Live?**

 - Jawaban model LLM: 
    - Relevan ✅ : Jawaban berada dalam konteks pertanyaan.
    - Kejelasan ✅ : Tutur bahasa yang digunakan nyambung dan dapat dimengerti manusia.
    - Kelengkapan ⚠️ : Sama seperti model sebelumnya kurang lengkap karena tidak menyebutkan teknisnya, seperti kartu lelang, dan penghitungan waktu.

    Jawaban model bersifat deskriptif umum dan promotif, tapi kehilangan bagian mekanisme teknis. Dari sisi CS (customer service), masih bisa dianggap cukup, namun tidak sekomprehensif data sebenarnya.

2. **Bagaimana cara mengisi saldo ShopeePay melalui SeaBank?**

 - Jawaban model LLM: 
    - Relevan ✅ : Relevan secara topik.
    - Kejelasan ✅ : Step-step yang diberikan jelas dan runut, serta mudah dipahami.
    - Kelengkapan ❌ : Step yang diberikan oleh `gpt3.5-turbo` berbeda dengan data actual, dimana data actual memulai step dengan membuka aplikasi SeaBank, sedangkan jawaban model memulai dari aplikasi Shopee.

    Model menjawab dengan percaya diri tapi salah prosedur. Di mana model menyusun jawaban yang terdengar logis tapi salah berdasarkan data aktual.

3. **Bisa ajari aku teknik bela diri?**

 - Jawaban model LLM: 
    - Relevan ✅ : Dengan sopan menolak menjawab karena tidak sesuai layanan.
    - Kejelasan ✅ : Singkat, padat, dan sesuai ekspektasi CS.

4. **Mengapa ayam menyebrang jalan?**

 - Jawaban model LLM: 
    - Relevan ✅ : Menolak dengan baik pertanyaan di luar konteks.
    - Kejelasan ✅ : Jelas.

**Query Lainnya Menggunakan GPT3.5**

In [14]:
question = 'Apa itu Shopee Mall?'
result = qa_gpt.invoke(question)
result

{'query': 'Apa itu Shopee Mall?',
 'result': 'Shopee Mall adalah fitur di Shopee yang menawarkan produk-produk dari brand resmi dan terpercaya. Produk-produk di Shopee Mall memiliki kualitas yang terjamin dan bisa diandalkan.'}

In [15]:
question = 'Duh kenapa sih saya harus bayar ongkos parkir driver shopee?'
result = qa_gpt.invoke(question)
result

{'query': 'Duh kenapa sih saya harus bayar ongkos parkir driver shopee?',
 'result': 'Maaf atas ketidaknyamanan yang Anda alami. Ongkos parkir driver Shopee merupakan biaya yang ditanggung oleh pelanggan. Hal ini dilakukan untuk memastikan driver dapat dengan mudah dan cepat mengantarkan pesanan Anda. Terima kasih atas pengertiannya.'}

In [16]:
question = 'Apakah paket COD bisa ditolak?'
result = qa_gpt.invoke(question)
result

{'query': 'Apakah paket COD bisa ditolak?',
 'result': 'Maaf, paket COD tidak bisa ditolak setelah pesanan dikonfirmasi. Jika ada masalah dengan pesanan COD, silakan hubungi layanan pelanggan kami untuk bantuan lebih lanjut. Terima kasih.'}

In [17]:
question = 'Bagaimana prosedur pengembalian barang?'
result = qa_gpt.invoke(question)
result

{'query': 'Bagaimana prosedur pengembalian barang?',
 'result': 'Untuk prosedur pengembalian barang, Anda dapat mengajukan permintaan pengembalian melalui aplikasi Shopee. Setelah permintaan disetujui, Anda dapat mengirimkan barang kembali ke penjual dan akan menerima pengembalian dana sesuai dengan kebijakan pengembalian Shopee. Jangan lupa untuk melampirkan bukti pengiriman saat mengirimkan barang kembali. Terima kasih.'}

In [18]:
question = 'Kapan Indonesia merdeka?'
result = qa_gpt.invoke(question)
result

{'query': 'Kapan Indonesia merdeka?',
 'result': 'Maaf, pertanyaan tersebut tidak terkait dengan layanan Shopee. Apakah ada pertanyaan lain yang bisa saya bantu?'}

In [19]:
question = 'Mengapa ayam menyebrang jalan?'
result = qa_gpt.invoke(question)
result

{'query': 'Mengapa ayam menyebrang jalan?',
 'result': 'Maaf, pertanyaan tersebut tidak terkait dengan layanan Shopee. Apakah ada pertanyaan lain yang bisa saya bantu?'}

In [20]:
question = 'Kamu tinggal dimana?'
result = qa_gpt.invoke(question)
result

{'query': 'Kamu tinggal dimana?',
 'result': 'Maaf, saya tidak bisa memberikan informasi pribadi saya. Bagaimana saya bisa membantu Anda dengan pertanyaan terkait Shopee?'}

# Kesimpulan

Berdasarkan hasil eksperimen QnA **positive case** dan **negative case** model `HuggingFaceH4/zephyr-7b-beta` dan `gpt-3.5-turbo`, terlihat bahwa masing-masing model memiliki kekuatan dan kelemahan tersendiri dalam konteks layanan pelanggan e-commerce.

**zephyr-7b-beta**
  - Kelebihan: Dapat memberikan jawaban yang relevan.
  - Kekurangan: Tidak dapat menjelaskan jawaban prosedural.

**gpt-3.5-turbo**
  - Kelebihan: Juga dapat membarikan jawaban yang relevan dan dapat menjelaskan jawaban yang membutuhkan prosedur tertentu.
  - Kekurangan: Ada prosedur yang tidak tepat pada jawaban no 2.

  Keduanya bisa dipilih sesuai kebutuhan: Zephyr untuk interaksi ringan, GPT untuk layanan CS yang sering kali membutuhkan jawaban prosedural.